### Objective

> Graph는 Node들의 연결체로 구성되어 있다. 텐서플로우 또한, Operation이 어떤 식으로 연결되어 있냐로 구성되어 있다. 이를 코드로서 파악해보자 

In [1]:
%matplotlib inline
import numpy as np
import tensorflow as tf

import matplotlib.pyplot as plt

from IPython.core.display import display, HTML 
display(HTML("<style>.container { width:100% !important;}</style>"))

In [2]:
######
# jupyter에서 바로 graph를 볼 수 있도록 만들어주는 helper Method
#######

from IPython.display import clear_output, Image, display, HTML
import numpy as np    

def strip_consts(graph_def, max_const_size=32):
    """Strip large constant values from graph_def."""
    strip_def = tf.GraphDef()
    for n0 in graph_def.node:
        n = strip_def.node.add() 
        n.MergeFrom(n0)
        if n.op == 'Const':
            tensor = n.attr['value'].tensor
            size = len(tensor.tensor_content)
            if size > max_const_size:
                tensor.tensor_content = "<stripped %d bytes>"%size
    return strip_def

def show_graph(graph_def, max_const_size=32):
    """Visualize TensorFlow graph."""
    if hasattr(graph_def, 'as_graph_def'):
        graph_def = graph_def.as_graph_def()
    strip_def = strip_consts(graph_def, max_const_size=max_const_size)
    code = """
        <script>
          function load() {{
            document.getElementById("{id}").pbtxt = {data};
          }}
        </script>
        <link rel="import" href="https://tensorboard.appspot.com/tf-graph-basic.build.html" onload=load()>
        <div style="height:600px">
          <tf-graph-basic id="{id}"></tf-graph-basic>
        </div>
    """.format(data=repr(str(strip_def)), id='graph'+str(np.random.rand()))

    iframe = """
        <iframe seamless style="width:1200px;height:620px;border:0" srcdoc="{}"></iframe>
    """.format(code.replace('"', '&quot;'))
    display(HTML(iframe))

# Tensorflow에서의 graph

> 텐서플로우의 graph는 `tf.Operation`(node)과 `tf.Tensor`(edge)로 이어져 있고, 이것들이 어떤 식으로 이어져 있는지를 기록되어 있다. `tf.Graph`는 하나의 Namespace 공간으로, 우리는 어떤 `tf.Graph` 아래에 `tf.Operation`을 추가시키는 지 결정할 수 있다. 

#### 우선 간단한 형태의 graph를 그려보자

In [3]:
# Build the graph
graph1 = tf.Graph() # graph를 만들기

with graph1.as_default():
    # 현재 Grpah Namespace를 graph1으로 설정
    # -> 아래 추가되는 모든 Operation은 graph1에 추가
    a = tf.constant(value=0.0, name='a')
    b = tf.constant(value=1.0, name='b')
    
    z = tf.add(a,b, name='plus')
show_graph(graph1)

# Run the graph
sess = tf.Session(graph=graph1)
print(sess.run(z))

1.0


위의 과정은 매우매우 단순하다. 하지만, 여기에는, Tensorflow의 핵심적인 이슈들이 담겨져 있다. 이것을 심도 깊게 한번 살펴보자. 

## Tensorflow의 제일 중요한 구성 요소, `Operation`

> Graph에서 모든 Node는 Operation이다. 즉, `tf.constant` 하나의 operation로, 상수를 generate하는 연산이다. 그러면 실제로 연산 결과는 어디에 담기는 것일까? 바로 `tensor`에 담긴다.

In [4]:
graph1 = tf.Graph() # graph를 만들기

with graph1.as_default():
    # graph1에 아래의 operation들을 추가
    a = tf.constant(value=0.0, name='a')
    b = tf.constant(value=1.0, name='b')
    
    z = tf.add(a,b, name='plus')

print(a) # tf.constant의 return 값은 Operation이 아니라 출력인 Tensor이다.

Tensor("a:0", shape=(), dtype=float32)


그렇다면 어떻게 graph에서 Node, 즉 `tf.constant`를 가져올 수 있을까? 아래와 같은 메소드를 이용하면 된다.

In [5]:
add_node = graph1.get_operation_by_name('plus')

add_node

<tf.Operation 'plus' type=Add>

#### 핵심은 바로 이 Operation에 있다.

Operation의 구조는 아래와 같다.

![](../../misc/tensorflow-basis/operation_structure.png)

* inputs : 해당 Operation의 입력값(Tensor) 
* outputs : 해당 Operation의 출력값(Tensor)
* node_def : 해당 Operation의 정의
* op_type : 해당 Operation의 연산 종류
* name : 해당 Operation의 이름. 고유해야 함


> 각각을 자세히 살펴보자

----

#### 1. node_def : 이 연산은 어떤 연산 타입인가? 

    * name : 해당 node의 이름 -> 이것은 graph마다 unique해야함! (중복 불가)
    * op : Operation의 타입 -> 이것은 이후 텐서플로우에서 이 타입에 해당하는 커널을 호출하는 식
    ````c++
    REGISTER_OP("Add")
    .Input("x: T")
    .Input("y: T")
    .Output("z: T")
    .Attr(
        "T: {bfloat16, half, float, double, uint8, int8, int16, int32, int64, "
        "complex64, complex128, string}")
    .SetShapeFn(shape_inference::BroadcastBinaryOpShapeFn);
    ````
    reference -> [tensorflow source code](https://github.com/tensorflow/tensorflow/blob/master/tensorflow/core/ops/math_ops.cc)
    * input : 해당 Operation에 필요한 input list
    
    

In [6]:
print(add_node.node_def)

name: "plus"
op: "Add"
input: "a"
input: "b"
attr {
  key: "T"
  value {
    type: DT_FLOAT
  }
}



-----

#### 2. inputs : 이 연산에 필요한 입력값(tensor)들이 무엇이 있는가? 

`add_node`에는 지금 `a`,`b` 두개의 입력값이 필요하다. 

In [7]:
print(list(add_node.inputs))

[<tf.Tensor 'a:0' shape=() dtype=float32>, <tf.Tensor 'b:0' shape=() dtype=float32>]


----

#### 3. outputs : 이 연산 직후, 어떤 출력값(tensor)들이 생성되는가? 

In [8]:
print(add_node.outputs)

[<tf.Tensor 'plus:0' shape=() dtype=float32>]


여기서 Tensor의 이름 명명 방식이 나온다. Tensor의 명명 방식은 <node_name>:<output_index>이다. 지금은 add_node의 outputs 중 0번째 출력 Tensor의 이름이 "plus:0"으로 나온다. 그럼 output이 2개 나오는 경우도 있을까? 가끔씩 있다. 예를 들어 아래의 `split` Operation을 확인해보자.

In [9]:
split_graph = tf.Graph()

with split_graph.as_default():
    X = tf.constant([[1,2,3],[2,3,4],[3,4,5]], name='X')
    split0, split1 = tf.split(X, [1,2], 1) # 1번째 축을 기준으로, (3, 1), (3, 2) 로 쪼갬
    
split_node = split_graph.get_operation_by_name('split')
split_node.outputs

[<tf.Tensor 'split:0' shape=(3, 1) dtype=int32>,
 <tf.Tensor 'split:1' shape=(3, 2) dtype=int32>]

위와 같은 경우 한 Operation에서 여러 개의 출력값이 나온다. 이런 경우을 위해, Tensor의 명명방식은 <Node_name>:<output_index>로 된다. 

----

4. device : 이 연산이 어느 device 아래에서 돌아가는가? 

In [10]:
add_node.device

''

> 지정하지 않으면, Tensorflow에서는 우선적으로 GPU에 배정하고, 
없으면 CPU에 배정하는 방식으로 지정한다.


In [11]:
graph2 = tf.Graph()
with graph2.as_default():
    with tf.device('/cpu:0'):
        a1 = tf.constant(value=0.0, name='cpu_a')
    with tf.device('/gpu:0'):
        a2 = tf.constant(value=0.0, name='gpu_a')
print(a1.device)
print(a2.device)

/device:CPU:0
/device:GPU:0


> 위와 같이 강제로 Device를 지정할 수 있고, 이럴 경우 꼭 Graph의 모든 operation이 같은 연산에 들어갈 필요가 없다. 예를 들어 매우 큰 모델을 만들경우, 모델의 특정 부분은 /gpu:0에서 처리하게 하고, 모델의 나머지 부분은 /gpu:1에서 처리하게 함으로써, 병렬 처리를 구현할 수도 있다.

-----

## Graph에 값을 집어 넣기, `Placeholder`

In [12]:
graph3 = tf.Graph()
with graph3.as_default():
    x = tf.placeholder(tf.float32, name='x')
    a = tf.constant(1., name='1')
    add_node = tf.add(x,a,name='add_1')

나중에 Session에 대해 다시 다룰 거지만, 아래와 같이, add_node에 연관된 placeholder는 꼭 feed_dict에 포함시켜야 한다.

In [13]:
with tf.Session(graph=graph3) as sess:
    print(sess.run(add_node, feed_dict={x:1}))
    print(sess.run(add_node, feed_dict={'x:0':1}))

2.0
2.0


가끔씩은 필요없는 경우가 있다. 예를들어, dropout에 꼭 들어가는 train인지 아닌지 정해주는
placeholder는 매번 코드에 넣기 번거롭다. 그런경우에는 `tf.placeholder_with_default`를 이용하면 깔끔하다. 

또 예를 들어, 나중에 Serving할 경우에, is_train에 인자를 넣어주는 행위는 굉장히 redundant한 코드로 변질되기 때문에, 처음부터 placeholder_with_default와 같은 코드로 제거해주는 것이 깔끔하다

In [14]:
graph3 = tf.Graph()
with graph3.as_default():
    is_train = tf.placeholder_with_default(False,None, name='is_train')
    X = tf.ones((1, 5,5,3),dtype=tf.float32,name='X')
    y = tf.layers.dropout(X, training=is_train, name='dropout')

In [15]:
with tf.Session(graph=graph3) as sess:
    print("without feed_dict")
    print(sess.run(y))
    print("with feed_dict(True)")
    print(sess.run(y,feed_dict={is_train:True}))

without feed_dict
[[[[1. 1. 1.]
   [1. 1. 1.]
   [1. 1. 1.]
   [1. 1. 1.]
   [1. 1. 1.]]

  [[1. 1. 1.]
   [1. 1. 1.]
   [1. 1. 1.]
   [1. 1. 1.]
   [1. 1. 1.]]

  [[1. 1. 1.]
   [1. 1. 1.]
   [1. 1. 1.]
   [1. 1. 1.]
   [1. 1. 1.]]

  [[1. 1. 1.]
   [1. 1. 1.]
   [1. 1. 1.]
   [1. 1. 1.]
   [1. 1. 1.]]

  [[1. 1. 1.]
   [1. 1. 1.]
   [1. 1. 1.]
   [1. 1. 1.]
   [1. 1. 1.]]]]
with feed_dict(True)
[[[[2. 0. 2.]
   [2. 0. 0.]
   [2. 2. 2.]
   [0. 2. 0.]
   [2. 2. 0.]]

  [[0. 0. 0.]
   [0. 0. 2.]
   [0. 0. 2.]
   [0. 0. 0.]
   [2. 2. 0.]]

  [[2. 2. 2.]
   [2. 2. 0.]
   [2. 0. 0.]
   [2. 2. 0.]
   [0. 2. 2.]]

  [[0. 0. 2.]
   [2. 0. 2.]
   [0. 2. 0.]
   [2. 0. 0.]
   [0. 0. 0.]]

  [[0. 2. 2.]
   [0. 0. 2.]
   [2. 2. 2.]
   [0. 0. 2.]
   [2. 2. 0.]]]]


## Graph 내 저장공간, `Variable`

> 텐서플로우에서는 Operation과 Tensor로 이루어져 있기 때문에, 기본적으로는 stateless, 즉 저장 공간이 따로 배치되어 있지 않다. 우리가 연산을 할 때에만, placeholder에 주입된 Tensor을 바탕으로 연산이 이루어지고, 그 외에는 메모리 공간에 남아있지 않다. 

> 하지만 딥러닝 모델에서 제일 중요한 것 중 하나는 바로 Weight이다. 이 Weight를 처리하기 위해, 별도의 객체가 있는데 그것이 바로 `Variable`이다. `Variable`은 메모리 공간이 있는 `Tensor`라고 생각하면 속편하다.(사실은 Operation으로 구성되어 있지만)

In [16]:
graph4 = tf.Graph()
with graph4.as_default():
    X = tf.placeholder(tf.float32, shape=(1,3), name='X')
    W = tf.Variable([[.5,1.,.5],
                     [1.,2.,.5],
                     [1.5,.5,1.]], name='weight')
    y = tf.matmul(X,W,name='matmul')

In [17]:
graph4.get_operations()

[<tf.Operation 'X' type=Placeholder>,
 <tf.Operation 'weight/initial_value' type=Const>,
 <tf.Operation 'weight' type=VariableV2>,
 <tf.Operation 'weight/Assign' type=Assign>,
 <tf.Operation 'weight/read' type=Identity>,
 <tf.Operation 'matmul' type=MatMul>]

아래를 보자. `tf.Variable`에 관련된 것은 늘 4개의 Operation이 자동으로 따라 붙는다.

1. `weight/initial_value` -> weight의 초기화 부분(create)
2. `weight` -> weight의 값이 저장된 부분(save)
3. `weight/Assign` -> weight의 값을 갱신하는 부분(update)
4. `weight/read` -> weight의 값을 읽어오는 부분(read)

In [18]:
# variable은 아래와 같은 방식으로 가져올 수 있다.
variables = graph4.get_collection(tf.GraphKeys.GLOBAL_VARIABLES)
variable = variables[0]
variable

<tf.Variable 'weight:0' shape=(3, 3) dtype=float32_ref>

In [19]:
with tf.Session(graph=graph4) as sess:
    variable.initializer.run() # 이것을 하지 않으면, 
                               # FailedPreconditionError가 발생
    print(sess.run(y,feed_dict={X:[[1,2,3]]}))

[[7.  6.5 4.5]]


#### `get_variable` vs `Variable`

> 묘하게 다른 구석이 바로 get_variable과 Variable. get_variable은 보통 공유할 수 있는 녀석이라고 생각하면 된다. 한번 보자

#### tf.Variable로 하기

In [20]:
graph4 = tf.Graph()
with graph4.as_default():
    X = tf.placeholder(tf.float32, shape=(1,3), name='X')
    
    # random 초기화
    normal_init = tf.random.normal(shape=(3,3))
    W1 = tf.Variable(normal_init, name='weight')
    y1 = tf.matmul(X,W1,name='matmul')

    normal_init = tf.random.normal(shape=(3,3))
    W2 = tf.Variable(normal_init, name='weight')
    y2 = tf.matmul(X,W2,name='matmul')
    
show_graph(graph4)

In [21]:
with tf.Session(graph=graph4) as sess:
    for variable in tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES):
        variable.initializer.run()
        
    print(sess.run(y1,feed_dict={X:[[1,2,3]]}))
    print(sess.run(y2,feed_dict={X:[[1,2,3]]}))

[[ 1.5795747 -3.565354   0.7005489]]
[[-1.8394544 -5.2051034  5.7503166]]


> `tf.Variable`의 경우, 이렇게 다른 결과가 나온다.

#### tf.get_variable

In [22]:
graph4 = tf.Graph()
with graph4.as_default():
    X = tf.placeholder(tf.float32, shape=(1,3), name='X')
    
    # random 초기화
    normal_init = tf.random.normal(shape=(3,3))
    with tf.variable_scope('A') as scope:
        W1 = tf.get_variable('weight', initializer=normal_init)
    y1 = tf.matmul(X,W1,name='matmul_1')
    
    with tf.variable_scope('A',reuse=True) as scope:
        # reuse를 하기 위해서는 꼭 variable_scope를 써야함...
        W2 = tf.get_variable('weight')
    y2 = tf.matmul(X,W2,name='matmul_2')
    
show_graph(graph4)

In [23]:
with tf.Session(graph=graph4) as sess:
    for variable in tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES):
        # 이것을 하지 않으면, FailedPreconditionError가 발생
        variable.initializer.run() 
    print(sess.run(y1,feed_dict={X:[[1,2,3]]}))
    print(sess.run(y2,feed_dict={X:[[1,2,3]]}))

[[-3.4953508 -1.6118951  8.292189 ]]
[[-3.4953508 -1.6118951  8.292189 ]]


> `get_collection`으로 weight를 공유하면, 이렇게 같은 결과가 나온다. 

## Graph을 보다 구조적으로 다루는 방법, `scope`와 `Collection`

우리가 다루는 딥러닝 모델들은 대부분 복잡하고 지저분하고 엉켜있기 마련이다. 이것을 좀 개선할 수 있는 방법이 바로 `scope`와 `Collection`이다. 이 코드를 잘 쓰느냐, 못 쓰느냐가 코드의 가독성에 매우 큰 영향을 미친다고 생각된다.

예를 들어 아래의 간단한 mnist 모델을 구현했다고 생각해보자. 

In [24]:
graph6 = tf.Graph()
n_inputs = 784
n_hidden1 = 100
n_hidden2 = 100
n_hidden3 = 100
n_outputs = 10

with graph6.as_default():
    X = tf.placeholder(tf.float32, (None,784))
    Y = tf.placeholder(tf.float32, (None,10))
    lr = tf.placeholder_with_default(0.01, None)
    
    #####################
    # Inference Model 부분
    # 은닉층이 3개인 Fully-connected Model
    #####################
    
    init = tf.initializers.glorot_normal()
    # 1번째 은닉층
    weight1 = tf.get_variable('hidden1',
                              shape=(n_inputs,n_hidden1),
                              initializer=init,)
    bias1 = tf.get_variable('bias1',
                            shape=(1,n_hidden1),
                            initializer=init)
    
    z1 = tf.matmul(X,weight1)+bias1
    a1 = tf.nn.relu(z1)
    
    # 2번째 은닉층
    weight2 = tf.get_variable('hidden2',
                              shape=(n_hidden1,n_hidden2),
                              initializer=init,)
    bias2 = tf.get_variable('bias2',
                            shape=(1,n_hidden2),
                            initializer=init)
    
    z2 = tf.matmul(a1,weight2)+bias2
    a2 = tf.nn.relu(z2)
    
    # 3번째 은닉층
    weight3 = tf.get_variable('hidden3',
                              shape=(n_hidden2,n_hidden3),
                              initializer=init,)
    bias3 = tf.get_variable('bias3',
                            shape=(1,n_hidden2),
                            initializer=init)
    
    z3 = tf.matmul(a2,weight3)+bias3
    a3 = tf.nn.relu(z3)
    
    # 출력층
    weight_out = tf.get_variable('weight_output',
                              shape=(n_hidden3,n_outputs),
                              initializer=init)
    bias_out = tf.get_variable('bias_output',
                            shape=(1,n_outputs),
                            initializer=init)
    logits = tf.matmul(a3, weight_out) + bias_out
    
    #####################
    # Loss Calculation 부분
    # Cross Entropy를 기준으로 계산
    #####################
    pred = tf.nn.softmax(logits)
    loss = tf.reduce_mean(-tf.reduce_sum(Y*tf.log(pred),
                                         reduction_indices=[1]))
    
    #####################
    # Metric Calculation 부분
    # 은닉층이 3개인 Fully-connected Model
    #####################
    pred_arg = tf.argmax(logits,axis=1)
    Y_arg = tf.argmax(Y,axis=1)
    correct = tf.reduce_sum(
                tf.cast(
                    tf.equal(pred_arg,Y_arg),
                    tf.float64)
              )
    acc = correct / tf.cast(tf.size(Y_arg),tf.float64)
    
    ######################
    # Training Operation 부분
    # 이 레이어에는 총 8개의 variable들이 있다. 
    # 각각 update해주어야 한다.
    ######################
    variables = [weight1,bias1,weight2,bias2,
                 weight3,bias3,weight_out,bias_out]
    grads = tf.gradients(loss, variables) 
    
    update_ops = []
    for variable, grad in zip(variables, grads):    
        update_ops.append(tf.assign_sub(variable, lr*grad))
    
    training_op = tf.group(update_ops)
    
show_graph(graph6)

In [25]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [26]:
n_epochs = 10
batch_size = 100 
n_batch = mnist.train.num_examples // batch_size

sess = tf.Session(graph=graph6)

with tf.Session(graph=graph6) as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(n_epochs):
        test_image = mnist.test.images
        test_label = mnist.test.labels
        loss_value, acc_value = sess.run([loss,acc],
                                         feed_dict={
                                             X:test_image,
                                             Y:test_label})
        print(f"{i} epoch | loss : {loss_value:.5f}, acc : {acc_value:.5f}")
        
        for _ in range(n_batch):
            batch_image, batch_label = mnist.train.next_batch(batch_size)
            sess.run(training_op,feed_dict={X:batch_image,
                                            Y:batch_label})
        


0 epoch | loss : 2.48756, acc : 0.10320
1 epoch | loss : 0.59574, acc : 0.84290
2 epoch | loss : 0.37629, acc : 0.89600
3 epoch | loss : 0.31478, acc : 0.91180
4 epoch | loss : 0.28730, acc : 0.91840
5 epoch | loss : 0.26451, acc : 0.92290
6 epoch | loss : 0.25352, acc : 0.92540
7 epoch | loss : 0.23758, acc : 0.93080
8 epoch | loss : 0.22163, acc : 0.93450
9 epoch | loss : 0.21537, acc : 0.93620


암만 봐도 너무 복잡하다. MNIST만 짜더라도, Low-level API로 짜면 이런 식으로 코드가 더럽다. 이것을 좀 더 구조적으로 짤 수 있도록 해보자. tensorflow에서 제공하는 tf.variable_scope와 tf.collection을 이용하면 좀 더 나은 방식으로 코드를 짤 수 있다.

In [27]:
graph6 = tf.Graph()
n_inputs = 784
n_hidden1 = 100
n_hidden2 = 100
n_hidden3 = 100
n_outputs = 10

with graph6.as_default():
    X = tf.placeholder(tf.float32, (None,784), name='input')
    Y = tf.placeholder(tf.float32, (None,10), name='label')
    lr = tf.placeholder_with_default(0.01, None, name='learning_rate')
    init = tf.initializers.glorot_normal()
    zero_init = tf.initializers.zeros()
    
    # 1번째 ~ 3번째 은닉층
    n_nums = [n_inputs, n_hidden1, n_hidden2, n_hidden3]
    
    a = X
    for i in range(1,4):
        # 변수와 Operation의 Namespace를 "hidden[]"으로 묶어줌
        with tf.variable_scope(f"hidden{i}"):
            weight = tf.get_variable('weight',
                                     shape=(n_nums[i-1],n_nums[i]),
                                     initializer=init,)
            bias = tf.get_variable('bias',
                                   shape=(1,n_nums[i]),
                                   initializer=zero_init)  
            # collection에 추가
            tf.add_to_collection(tf.GraphKeys.WEIGHTS, weight)
            tf.add_to_collection(tf.GraphKeys.BIASES, bias)
            
            z = tf.matmul(a,weight)+bias
            a = tf.nn.relu(z,name='activation')
            
            tf.add_to_collection(tf.GraphKeys.ACTIVATIONS, a)
        
    # 출력층
    with tf.variable_scope('output'):
        # 변수와 Operation의 Namespace를 "output"으로 묶어줌
        weight = tf.get_variable('weight',
                                 shape=(n_hidden3,n_outputs),
                                 initializer=init)
        bias = tf.get_variable('bias',
                               shape=(1,n_outputs),
                               initializer=zero_init)
        
        # collection에 추가
        tf.add_to_collection(tf.GraphKeys.WEIGHTS, weight)
        tf.add_to_collection(tf.GraphKeys.BIASES, bias)
 
        logits = tf.matmul(a, weight) + bias
        logits = tf.identity(logits, name="logit")
        pred = tf.nn.softmax(logits, name='prediction')        
        tf.add_to_collection(tf.GraphKeys.ACTIVATIONS, pred)
    
    with tf.variable_scope('loss'):
        loss = tf.reduce_mean(-tf.reduce_sum(Y*tf.log(pred),
                                             reduction_indices=[1]))
        # collection에 추가
        tf.add_to_collection(tf.GraphKeys.LOSSES, loss)
    
    with tf.variable_scope('metric'):
        pred_arg = tf.argmax(logits,axis=1)
        Y_arg = tf.argmax(Y,axis=1)
        correct = tf.reduce_sum(
                    tf.cast(
                        tf.equal(pred_arg,Y_arg),
                        tf.float64)
                  )
        acc = correct / tf.cast(tf.size(Y_arg),tf.float64)
        # collection에 추가
        tf.add_to_collection(tf.GraphKeys.METRIC_VARIABLES, acc)
    
    # variables에는 모든 wegiths들이 담겨져 있음.
    weights = tf.get_collection(tf.GraphKeys.WEIGHTS)
    biases = tf.get_collection(tf.GraphKeys.BIASES)
    
    with tf.variable_scope('SGD'):
        weight_grad = tf.gradients(loss, weights,name='weight-gradient')
        bias_grad = tf.gradients(loss, biases,name='bias-gradient') 
        variables = weights + biases
        grads = weight_grad + bias_grad

        update_ops = []
        for variable, grad in zip(variables, grads):    
            update_ops.append(tf.assign_sub(variable, lr*grad))
        
        training_op = tf.group(update_ops, name='training_op')
        tf.add_to_collection(tf.GraphKeys.TRAIN_OP, training_op)
    
show_graph(graph6)

> 찍어보면, 훨씬 더 가독성이 좋은 graph가 그려지는 것을 알 수 있다. 계산이 어떤식으로 동작하는지가 훨씬 더 명료해진다. 이것은 `tf.variable_scope`로 묶어주었기 때문이다.

> collection은 Graph의 메타 정보이다. 우리는 수많은 `Operation`, `Tensor`, `Variable`들을 관리하기가 매우매우 어렵다. 이것을 보다 Semantic하게, 의미에 맞게 관리하는 방법이 바로 `Collection`이다. Collection은 기본적으로는 `tf.GraphKeys`라는 키값 정보를 중심으로 돌아가는데, `tf.GraphKeys`에는 아래와 같은 것들이 있다.

In [28]:
[key for key in dir(tf.GraphKeys) if key[0] != "_"] # 기본 키값 요소들

['ACTIVATIONS',
 'ASSET_FILEPATHS',
 'BIASES',
 'CONCATENATED_VARIABLES',
 'COND_CONTEXT',
 'EVAL_STEP',
 'GLOBAL_STEP',
 'GLOBAL_VARIABLES',
 'INIT_OP',
 'LOCAL_INIT_OP',
 'LOCAL_RESOURCES',
 'LOCAL_VARIABLES',
 'LOSSES',
 'METRIC_VARIABLES',
 'MODEL_VARIABLES',
 'MOVING_AVERAGE_VARIABLES',
 'QUEUE_RUNNERS',
 'READY_FOR_LOCAL_INIT_OP',
 'READY_OP',
 'REGULARIZATION_LOSSES',
 'RESOURCES',
 'SAVEABLE_OBJECTS',
 'SAVERS',
 'SUMMARIES',
 'SUMMARY_OP',
 'TABLE_INITIALIZERS',
 'TRAINABLE_RESOURCE_VARIABLES',
 'TRAINABLE_VARIABLES',
 'TRAIN_OP',
 'UPDATE_OPS',
 'VARIABLES',
 'WEIGHTS',
 'WHILE_CONTEXT']

#### 이름 부터 느낌이 보통 올것이다. 

In [29]:
graph6.get_collection(tf.GraphKeys.WEIGHTS) # kernel weight에 해당하는 녀석들만 가져올 수 있고,

[<tf.Variable 'hidden1/weight:0' shape=(784, 100) dtype=float32_ref>,
 <tf.Variable 'hidden2/weight:0' shape=(100, 100) dtype=float32_ref>,
 <tf.Variable 'hidden3/weight:0' shape=(100, 100) dtype=float32_ref>,
 <tf.Variable 'output/weight:0' shape=(100, 10) dtype=float32_ref>]

In [30]:
graph6.get_collection(tf.GraphKeys.BIASES) # bias에 해당하는 녀석들만 가져올 수 있다.

[<tf.Variable 'hidden1/bias:0' shape=(1, 100) dtype=float32_ref>,
 <tf.Variable 'hidden2/bias:0' shape=(1, 100) dtype=float32_ref>,
 <tf.Variable 'hidden3/bias:0' shape=(1, 100) dtype=float32_ref>,
 <tf.Variable 'output/bias:0' shape=(1, 10) dtype=float32_ref>]

In [31]:
graph6.get_collection(tf.GraphKeys.LOSSES) # 이렇게 로스 function을 가져올 수 있고

[<tf.Tensor 'loss/Mean:0' shape=() dtype=float32>]

In [32]:
graph6.get_collection(tf.GraphKeys.TRAIN_OP) # 이렇게 training operation을 가져올 수 있고

[<tf.Operation 'SGD/training_op' type=NoOp>]

In [33]:
# 만약 우리가 각층의 Activation을 찍고 싶다고 생각이 들면, 아래처럼 Tensor들을 가져와서 돌려볼 수도 있다. 
graph6.get_collection(tf.GraphKeys.ACTIVATIONS)

[<tf.Tensor 'hidden1/activation:0' shape=(?, 100) dtype=float32>,
 <tf.Tensor 'hidden2/activation:0' shape=(?, 100) dtype=float32>,
 <tf.Tensor 'hidden3/activation:0' shape=(?, 100) dtype=float32>,
 <tf.Tensor 'output/prediction:0' shape=(?, 10) dtype=float32>]

> 이렇게 Collection으로 Graph의 많은 요소들을 Semantic하게 묶기 시작하면, 좀 더 깔끔하게 주요 요소 별로 분리하여 서술할 수 있게 된다.

In [34]:
def build_inference_model(n_inputs=784,n_hidden1=100,n_hidden2=100,n_hidden3=100,n_outputs=10):
    """ MNIST의 순전파 부분에 관련된 Model.
    아래 코드는 이 모델에 inference할 때 거치는 Operation만 포함되어 있음
        
    """
    graph = tf.Graph()
    with graph.as_default():
        X = tf.placeholder(tf.float32, (None,784), name='input')
        init = tf.initializers.glorot_normal()
        zero_init = tf.initializers.zeros()

        # 1번째 ~ 3번째 은닉층
        n_nums = [n_inputs, n_hidden1, n_hidden2, n_hidden3]

        a = X
        for i in range(1,4):
            # 변수와 Operation의 Namespace를 "hidden[]"으로 묶어줌
            with tf.variable_scope(f"hidden{i}"):
                weight = tf.get_variable('weight',
                                         shape=(n_nums[i-1],n_nums[i]),
                                         initializer=init,)
                bias = tf.get_variable('bias',
                                       shape=(1,n_nums[i]),
                                       initializer=zero_init)  
                # collection에 추가
                tf.add_to_collection(tf.GraphKeys.WEIGHTS, weight)
                tf.add_to_collection(tf.GraphKeys.BIASES, bias)

                z = tf.matmul(a,weight)+bias
                a = tf.nn.relu(z,name='activation')

                tf.add_to_collection(tf.GraphKeys.ACTIVATIONS, a)

        # 출력층
        with tf.variable_scope('output'):
            # 변수와 Operation의 Namespace를 "output"으로 묶어줌
            weight = tf.get_variable('weight',
                                     shape=(n_hidden3,n_outputs),
                                     initializer=init)
            bias = tf.get_variable('bias',
                                   shape=(1,n_outputs),
                                   initializer=zero_init)

            # collection에 추가
            tf.add_to_collection(tf.GraphKeys.WEIGHTS, weight)
            tf.add_to_collection(tf.GraphKeys.BIASES, bias)

            logits = tf.matmul(a, weight) + bias
            logits = tf.identity(logits, name="logit")
            pred = tf.nn.softmax(logits, name='prediction')        
            tf.add_to_collection(tf.GraphKeys.ACTIVATIONS, pred)
    return graph

def attach_loss_to_graph(graph):
    """
    graph에 해당하는 
    """
    with graph.as_default():
        Y = tf.placeholder(tf.float32, (None,10), name='label')
        pred = tf.get_collection(tf.GraphKeys.ACTIVATIONS)[-1]
        with tf.variable_scope('loss'):
            loss = tf.reduce_mean(-tf.reduce_sum(Y*tf.log(pred),
                                                 reduction_indices=[1]))
        # collection에 추가
        tf.add_to_collection(tf.GraphKeys.LOSSES, loss)
    return graph

def attach_training_op_to_graph(graph):
    """
    """
    with graph.as_default():
        lr = tf.placeholder_with_default(0.01, None, name='learning_rate')
        weights = tf.get_collection(tf.GraphKeys.WEIGHTS)
        biases = tf.get_collection(tf.GraphKeys.BIASES)
        loss = tf.get_collection(tf.GraphKeys.LOSSES)
        
        with tf.variable_scope('SGD'):
            weight_grad = tf.gradients(loss, weights,name='weight-gradient')
            bias_grad = tf.gradients(loss, biases,name='bias-gradient') 

            variables = weights + biases
            grads = weight_grad + bias_grad
            update_ops = []
            for variable, grad in zip(variables, grads):    
                update_ops.append(tf.assign_sub(variable, lr*grad))

            training_op = tf.group(update_ops, name='training_op')
            tf.add_to_collection(tf.GraphKeys.TRAIN_OP, training_op)   
    return graph
    
def attach_metrics_to_graph(graph):
    with graph.as_default():
        pred = tf.get_collection(tf.GraphKeys.ACTIVATIONS)[-1]
        Y = graph.get_tensor_by_name('label:0')
        
        with tf.variable_scope('metric'):
            pred_arg = tf.argmax(pred, axis=1)
            Y_arg = tf.argmax(Y,axis=1)
            correct = tf.reduce_sum(
                        tf.cast(
                            tf.equal(pred_arg,Y_arg),
                            tf.float64)
                      )
            acc = correct / tf.cast(tf.size(Y_arg),tf.float64)
            # collection에 추가
            tf.add_to_collection(tf.GraphKeys.METRIC_VARIABLES, acc)
    return graph

In [35]:
graph = build_inference_model()
#show_graph(graph)
graph = attach_loss_to_graph(graph)
#show_graph(graph)
graph = attach_training_op_to_graph(graph)
# show_graph(graph)
graph = attach_metrics_to_graph(graph)
show_graph(graph)

> 이러한 Collection은 Low-API Tensorflow에서는 직접 다 추가해야해서 귀찮지만, 실제로 High-API Tensorflow에서는 이미 다 추가되어 있다. 우리가 일일히 Collection에 지정하지 않더라도 자동으로 Collection에 포함시킨다.

In [36]:
graph6 = tf.Graph()
n_inputs = 784
n_hidden1 = 100
n_hidden2 = 100
n_hidden3 = 100
n_outputs = 10

with graph6.as_default():
    X = tf.placeholder(tf.float32, (None,784), name='input')
    Y = tf.placeholder(tf.float32, (None,10), name='label')
    lr = tf.placeholder_with_default(0.01, None, name='learning_rate')
    init = tf.initializers.glorot_normal()
    
    # 1번째 ~ 3번째 은닉층
    n_nums = [n_hidden1, n_hidden2, n_hidden3]
    a = X
    for i, units in enumerate([n_hidden1, n_hidden2, n_hidden3]):
        with tf.variable_scope(f'hidden{i+1}'):
            a = tf.layers.dense(a, units, 
                                activation=tf.nn.relu,
                                kernel_initializer=init)
        
    logit = tf.layers.dense(a,n_outputs,kernel_initializer=init)

    loss = tf.losses.softmax_cross_entropy(Y,logit)
    
    with tf.variable_scope('metric'):
        labels = tf.argmax(Y, 1)
        correct = tf.nn.in_top_k(logit, labels, 1)
        accuracy = tf.reduce_mean(tf.cast(correct, tf.float32)) 
        
        tf.add_to_collection(tf.GraphKeys.METRIC_VARIABLES, accuracy)
    
    training_op = tf.train.GradientDescentOptimizer(lr).minimize(loss)
show_graph(graph6)

In [37]:
graph6.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES)

[<tf.Variable 'hidden1/dense/kernel:0' shape=(784, 100) dtype=float32_ref>,
 <tf.Variable 'hidden1/dense/bias:0' shape=(100,) dtype=float32_ref>,
 <tf.Variable 'hidden2/dense/kernel:0' shape=(100, 100) dtype=float32_ref>,
 <tf.Variable 'hidden2/dense/bias:0' shape=(100,) dtype=float32_ref>,
 <tf.Variable 'hidden3/dense/kernel:0' shape=(100, 100) dtype=float32_ref>,
 <tf.Variable 'hidden3/dense/bias:0' shape=(100,) dtype=float32_ref>,
 <tf.Variable 'dense/kernel:0' shape=(100, 10) dtype=float32_ref>,
 <tf.Variable 'dense/bias:0' shape=(10,) dtype=float32_ref>]

In [38]:
graph6.get_collection(tf.GraphKeys.LOSSES)

[<tf.Tensor 'softmax_cross_entropy_loss/value:0' shape=() dtype=float32>]

In [39]:
graph6.get_collection(tf.GraphKeys.TRAIN_OP)

[<tf.Operation 'GradientDescent' type=NoOp>]

알아서 추가되는 것이 있고 없는 것이 있는데, `tf.GraphKeys.ACTIVATIONS`나 `tf.GraphKeys.WEIGHTS`은 알아서 추가해줘야 하는 메소드. 하지만 WEIGHTS의 경우에는 다른 방법이 존재한다. 

In [40]:
weights = graph6.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES,'^[\w//]+/kernel:0')
weights

[<tf.Variable 'hidden1/dense/kernel:0' shape=(784, 100) dtype=float32_ref>,
 <tf.Variable 'hidden2/dense/kernel:0' shape=(100, 100) dtype=float32_ref>,
 <tf.Variable 'hidden3/dense/kernel:0' shape=(100, 100) dtype=float32_ref>,
 <tf.Variable 'dense/kernel:0' shape=(100, 10) dtype=float32_ref>]

In [41]:
biases = graph6.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES,'^[\w//]+/bias:0')
biases

[<tf.Variable 'hidden1/dense/bias:0' shape=(100,) dtype=float32_ref>,
 <tf.Variable 'hidden2/dense/bias:0' shape=(100,) dtype=float32_ref>,
 <tf.Variable 'hidden3/dense/bias:0' shape=(100,) dtype=float32_ref>,
 <tf.Variable 'dense/bias:0' shape=(10,) dtype=float32_ref>]

<hr>

Copyright(c) 2019 by Public AI. All rights reserved.<br>
Writen by PAI, SangJae Kang ( rocketgrowthsj@publicai.co.kr )  last updated on 2019/01/24
<hr>